jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [12]:
import sklearn
import pandas as pd
import sqlite3
import json
import os
from sqlalchemy import create_engine


DATA_DIR = "./data"
DATA_FILE = os.path.join(DATA_DIR, "data.json")

store_columns = (
    "id",  # 음식점 고유번호
    "store_name",  # 음식점 이름
    "branch",  # 음식점 지점 여부
    "area",  # 음식점 위치
    "tel",  # 음식점 번호
    "address",  # 음식점 주소
    "latitude",  # 음식점 위도  # 위치기반 음식점 추천할 때 사용
    "longitude",  # 음식점 경도  # 위치기반 음식점 추천할 때 사용
    "category",  # 음식점 카테고리
    "review_cnt",  # 리뷰 갯수
)

menu_colums = (
    "id", # 메뉴 고유번호 
    "store",  # 음식점 고유번호 (FK)
    "menu_name",  # 메뉴 이름
    "price",  # 메뉴 가격
)

hour_colums = (
    "id", # 시간테이블 고유번호
    "oper_type",  # 영업시간 종류
    "week_type",  # 주단위 종류
    "store",  # 음식점 고유번호
    "mon",  # 월요일 포함유무
    "tue",  # 화요일 포함유무
    "wed",  # 수요일 포함유무
    "thu",  # 목요일 포함유무
    "fri",  # 금요일 포함유무
    "sat",  # 토요일 포함유무
    "sun",  # 일요일 포함유무
    "start_time",  # 시작시간
    "end_time",  # 종료시간
    "etc",  # 기타
)

review_columns = (
    "id",  # 리뷰 고유번호
    "store",  # 음식점 고유번호
    "user",  # 유저 고유번호
    "score",  # 평점
    "content",  # 리뷰 내용
    "reg_time",  # 리뷰 등록 시간
)

user_colums = (
    "id",  # 유저 고유번호
    "gender",  # 성별
    "born_year"  # 태어난 해
)

def import_data(data_path=DATA_FILE):
    try:
        with open(data_path, encoding="utf-8") as f:
                data = json.loads(f.read())
    except FileNotFoundError as e:
            print(f"`{data_path}` 가 존재하지 않습니다.")
            exit(1)

    stores = []  # 음식점 테이블
    menus = []  # 메뉴 테이블
    hours = []  # 음식점 시간 테이블
    reviews = []  # 리뷰 테이블
    users = []  # 유저 테이블

    idx = 0
    idx2 = 0
    for d in data:

        categories = [c["category"] for c in d["category_list"]]
        # review_cnt = 
        stores.append(
            [
                d["id"],
                d["name"],
                d["branch"],
                d["area"],
                d["tel"],
                d["address"],
                d["latitude"],
                d["longitude"],
                "|".join(categories),
                d["review_cnt"],
            ]
        )

        for menu in d["menu_list"]:
            idx += 1
            menus.append(
                [str(idx), d["id"], menu["menu"], menu["price"]]
            )
        
        for hour in d["bhour_list"]:
            idx2 += 1
            hours.append(
                [   str(idx2),
                    hour["type"], hour["week_type"], d["id"],
                    hour["mon"], hour["tue"], hour["wed"], hour["thu"],
                    hour["fri"], hour["sat"], hour["sun"],
                    hour["start_time"], hour["end_time"], hour["etc"]
                ]
            )
  
        for review in d["review_list"]:
            r = review["review_info"]
            u = review["writer_info"]

            reviews.append(
                [r["id"], d["id"], u["id"], r["score"], r["content"], r["reg_time"]]
            )

            users.append(
                [u["id"], u["gender"], u["born_year"]]
            )

    store_frame = pd.DataFrame(data=stores, columns=store_columns)
    menu_frame = pd.DataFrame(data=menus, columns=menu_colums)
    hour_frame = pd.DataFrame(data=hours, columns=hour_colums)
    review_frame = pd.DataFrame(data=reviews, columns=review_columns)
    user_frame = pd.DataFrame(data=users, columns=user_colums)
    user_frame = user_frame.drop_duplicates()

    return {"stores": store_frame, "menus": menu_frame, "hours": hour_frame,
            "reviews": review_frame, "users": user_frame}



data = import_data()


In [2]:
review_contents = data["reviews"]["content"]

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer().fit(review_contents)
tfidf_matrix = tfidfv.transform(review_contents).toarray()
print(tfidf_matrix) # 각 단어의 빈도수 기록
tdidf_vocab = tfidfv.vocabulary_
# print(tdidf_vocab) # 각 단어의 인덱스
print(tfidf_matrix.shape) # 91398 contents를 표현하기 위해 290794의 단어가 사용되었음

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(91398, 290794)


In [ ]:
# 코사인 유사도를 활용해서 문서의 유사도 구하기
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

### FOR LSA (tf-idf와 같은 단어 빈도수 이용한 수치화방법이 단어의 의미를 고려하지 못하므로 사용)

### LSA -> LDA 로 보완

* TF-IDF 행렬에 절단된 SVD 를 사용해 차원을 축소시켜, 단어들의 잠재적인 의미를 끌어냄
* 텍스트 전처리 후 geneism 사용해 LDA적용

In [36]:
# 1. 텍스트 전처리 - KoNLPy 라이브러리 활용해 형태소 분석 및 품사 태깅 (띄어쓰기 알고리즘, 정규화 이용해 맞춤법 틀린 문장도 어느정도 고쳐주며
# 형태소 분석과 품사 태깅을 해주는 여러 클래스 제공함)

In [38]:
text = pd.DataFrame(data["reviews"]["content"], columns=["content"])
text

,content
0,전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!
1,샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천
2,홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...
3,"전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건..."
4,친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...
5,"청국장갖다주며""보약이야""\n계란탕놓으며""미세먼지에좋아""\n평소불친절하다소문날정도로무..."
6,집밥을 향해 돌진하는 방배 마을 회관 같은 식당. 언제든 들려 한 끼 푸짐한 밥을 ...
7,연희동의 숨은 맛집. 재료의 신선함에 푸짐한 양까지 놓치지 않는다.\n버섯의 쫄깃함...
8,국물닭갈비가 흔치 않은데 대학교 근처라 가격도 저렴하고 세트구성도 있어서 학생들이 ...
9,


In [39]:
text.to_csv("content_db", mode="w")